## Sommaire: <a class="anchor" id="sommaire"></a>
* [Sommaire](#sommaire)
* [Package Loading](#package)
* [Model selection](#model_selection)
    * [FD001](#fd001)
    * [FD002](#fd002)
    * [FD003](#fd003)
    * [FD004](#fd004)

## Package's loading <a id='package'></a>

In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import warnings
import tensorflow as tf
import seaborn as sns
import sklearn
import random
import math
import time



from lime.lime_tabular import RecurrentTabularExplainer
from tqdm import tqdm
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, r2_score 
from sklearn.model_selection import GroupKFold
from sklearn import preprocessing
from keras import backend as K
from sklearn.preprocessing import MinMaxScaler , StandardScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation, GRU
from scipy import optimize
from tensorflow.keras import optimizers


from sp_modif.model_function import *
from sp_modif.methods import *
from sp_modif.data_prep import *
from sp_modif.evaluator import *
from sp_modif.SHAP import *
from sp_modif.L2X import *
from methods import *

%matplotlib inline
warnings.filterwarnings('ignore')

SEED = 0
def set_seed(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(SEED)
    random.seed(SEED)
    np.random.seed(SEED)
    tf.random.set_seed(SEED)

# Appeler la fonction pour fixer le seed
set_seed(SEED)

### FD004 <a class="anchor" id="fd004"></a>

In [2]:
# Load data
# Load data and preprocess
with tf.device('/device:GPU:0'):
    train, test, y_test = prepare_data('FD004.txt')
    print(train.shape, test.shape, y_test.shape)
    sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

    remaining_sensors = ["Ps30", "Nf", "phi", "BPR", "farB"] # selection based on main_notebook

    drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

rul_piecewise = 120
train['RUL'].clip(upper=rul_piecewise, inplace=True)

(61249, 27) (41214, 26) (248, 1)


In [3]:
# 1layers
def model_lstm_1layer(input_shape, nodes_per_layer, dropout, activation):
    
    cb = keras.callbacks.EarlyStopping(monitor='loss', patience=4)
    model = Sequential()
    model.add(LSTM(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape))
    model.add(Dropout(dropout))
    model.add(Dense(256))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',
                  optimizer=Adam(learning_rate=0.001))
    # model.save_weights(weights_file)

    return model

In [4]:
%%time

# RMSE	std_RMSE	S_score	std_S_score	MSE	std_MSE	nodes	dropout	activation	batch_size	TW
# 1	15.449126	0.0	1.242658e+03	0.0	238.675491	0.0	[64]	0.2	tanh	32	40
# RMSE	std_RMSE	S_score	std_S_score	MSE	std_MSE	alpha	nodes	dropout	activation	batch_size	TW
# 2	16.100150	0.0	1.752538e+03	0.0	259.214813	0.0	0.30	[64]	0.2	tanh	32	40
results = pd.DataFrame()

for SEED in range(5):  
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    # 0.20	[64]	0.3	tanh	32	25
    
    # parameter's sample
    weights_file = "weights_file_lstm_optimalmodel_clv.h5"
    alpha = 0.3
    sequence_length = 40
    epochs = 20
    nodes_per_layer = [64]
    dropout = 0.2
    activation = 'tanh'
    batch_size = 32
    remaining_sensors = remaining_sensors
    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout, activation)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)
            
    # Model fitting
    with tf.device('/device:GPU:0'):
        start_time = time.time()
        weights_file = model.get_weights()
        model.set_weights(weights_file)  # reset optimizer and node weights before every training iteration
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        end_time = time.time()
        
        training_time = end_time - start_time
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size, 'TW' : sequence_length, 
         'time': training_time}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results = pd.concat([results, pd.DataFrame(d, index=[0])], ignore_index=True)
    results.to_csv('results/base/fd004.csv')


(51538, 40, 5) (51538, 1) (248, 40, 5)
Epoch 1/20

1611/1611 [==============================] - 10s 6ms/step - loss: 1015.2298 - val_loss: 1246.5740
Epoch 2/20
1611/1611 [==============================] - 8s 5ms/step - loss: 455.8517 - val_loss: 1080.9202
Epoch 3/20
1611/1611 [==============================] - 9s 5ms/step - loss: 375.6517 - val_loss: 647.6152
Epoch 4/20
1611/1611 [==============================] - 9s 5ms/step - loss: 311.6118 - val_loss: 503.3330
Epoch 5/20
1611/1611 [==============================] - 9s 5ms/step - loss: 292.3782 - val_loss: 393.4156
Epoch 6/20
1611/1611 [==============================] - 9s 5ms/step - loss: 274.8222 - val_loss: 386.3050
Epoch 7/20
1611/1611 [==============================] - 9s 5ms/step - loss: 260.6937 - val_loss: 391.2215
Epoch 8/20
1611/1611 [==============================] - 9s 6ms/step - loss: 245.6545 - val_loss: 435.1175
Epoch 9/20
1611/1611 [==============================] - 9s 6ms/step - loss: 236.1186 - val_loss: 370.4720
E

In [5]:
results

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size,TW,time
0,18.902430,0.0,2310.933325,0.0,357.301849,0.0,[64],0.2,tanh,32,40,179.232490
1,14.837874,0.0,1377.840255,0.0,220.162506,0.0,[64],0.2,tanh,32,40,169.208207
2,18.856868,0.0,2858.685112,0.0,355.581482,0.0,[64],0.2,tanh,32,40,164.752809
3,16.112819,0.0,1236.360029,0.0,259.622955,0.0,[64],0.2,tanh,32,40,165.653482
4,18.711384,0.0,5717.449215,0.0,350.115906,0.0,[64],0.2,tanh,32,40,349.229202


### FD002 <a class="anchor" id="fd002"></a>

In [6]:
# Load data
# Load data and preprocess
train, test, y_test = prepare_data('FD002.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']
remaining_sensors = ["Ps30", "Nf", "phi", "epr"] # selection based on main_notebook [1] "Nf" [1] "Ps30" [1] "phi"
drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

rul_piecewise = 125
train['RUL'].clip(upper=rul_piecewise, inplace=True)

(53759, 27) (33991, 26) (259, 1)


In [7]:
%%time

# RMSE	std_RMSE	S_score	std_S_score	MSE	std_MSE	alpha	nodes	dropout	activation	batch_size	TW
# 16.603718	0.0	1.343367e+03	0.0	275.683502	0.0	0.20	[32]	0.1	tanh	128	40
results002 = pd.DataFrame()

for SEED in range(5):  
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    # 0.20	[64]	0.3	tanh	32	25
    
    # parameter's sample
    # weights_file = "weights_file_lstm_optimalmodel_clv.h5"
    alpha = 0.2
    sequence_length = 40
    epochs = 20
    nodes_per_layer = [32]
    dropout = 0.1
    activation = 'tanh'
    batch_size = 128
    remaining_sensors = remaining_sensors
    # create model
    input_shape = (sequence_length, len(remaining_sensors))    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout, activation)
       
    # Model fitting
    with tf.device('/device:GPU:0'):
        start_time = time.time()
        weights_file = model.get_weights()
        model.set_weights(weights_file)  # reset optimizer and node weights before every training iteration
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        end_time = time.time()
        
        training_time = end_time - start_time
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size, 'TW' : sequence_length, 
         'time': training_time}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results002 = pd.concat([results002, pd.DataFrame(d, index=[0])], ignore_index=True)
    results002.to_csv('results/base/fd002.csv')

(43619, 40, 4) (43619, 1) (259, 40, 4)
Epoch 1/20
341/341 [==============================] - 4s 9ms/step - loss: 2236.6396 - val_loss: 860.0703
Epoch 2/20
341/341 [==============================] - 3s 8ms/step - loss: 632.3809 - val_loss: 335.7626
Epoch 3/20
341/341 [==============================] - 3s 8ms/step - loss: 446.0878 - val_loss: 323.0473
Epoch 4/20
341/341 [==============================] - 3s 9ms/step - loss: 406.7737 - val_loss: 347.9665
Epoch 5/20
341/341 [==============================] - 3s 8ms/step - loss: 380.7448 - val_loss: 302.5203
Epoch 6/20
341/341 [==============================] - 3s 9ms/step - loss: 368.7535 - val_loss: 318.2314
Epoch 7/20
341/341 [==============================] - 3s 9ms/step - loss: 360.7570 - val_loss: 289.4062
Epoch 8/20
341/341 [==============================] - 3s 8ms/step - loss: 352.9235 - val_loss: 283.3463
Epoch 9/20
341/341 [==============================] - 4s 11ms/step - loss: 341.6447 - val_loss: 286.5387
Epoch 10/20
341/341 [==

In [8]:
results002

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size,TW,time
0,16.742477,0.0,1304.576739,0.0,280.310547,0.0,[32],0.1,tanh,128,40,108.490979
1,19.041899,0.0,8248.457896,0.0,362.593933,0.0,[32],0.1,tanh,128,40,153.369488
2,17.635494,0.0,1783.964319,0.0,311.010651,0.0,[32],0.1,tanh,128,40,174.991366
3,17.093295,0.0,1889.205625,0.0,292.180725,0.0,[32],0.1,tanh,128,40,152.966745
4,17.384912,0.0,1440.336592,0.0,302.235168,0.0,[32],0.1,tanh,128,40,154.377419


### FD003 <a class="anchor" id="fd003"></a>

In [9]:
# Load data
# Load data and preprocess
train, test, y_test = prepare_data('FD003.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']
remaining_sensors = ["Ps30", "NRc", "W31", "BPR"] # selection based on main_notebook [1] "Nf" [1] "Ps30" [1] "phi"
drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

rul_piecewise = 125
train['RUL'].clip(upper=rul_piecewise, inplace=True)

(24720, 27) (16596, 26) (100, 1)


In [10]:
%%time

# 	RMSE	std_RMSE	S_score	std_S_score	MSE	std_MSE	alpha	nodes	dropout	activation	batch_size	TW
# 43	15.404917	0.0	8.255290e+02	0.0	237.311462	0.0	0.10	[64]	0.2	tanh	32	35
results003 = pd.DataFrame()

for SEED in range(5):  
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    # 0.20	[64]	0.3	tanh	32	25
    
    # parameter's sample
    # weights_file = "weights_file_lstm_optimalmodel_clv.h5"
    alpha = 0.1
    sequence_length = 35
    epochs = 20
    nodes_per_layer = [64]
    dropout = 0.2
    activation = 'tanh'
    batch_size = 32
    remaining_sensors = remaining_sensors
    # create model
    input_shape = (sequence_length, len(remaining_sensors))    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout, activation)
       
    # Model fitting
    with tf.device('/device:GPU:0'):
        start_time = time.time()
        weights_file = model.get_weights()
        model.set_weights(weights_file)  # reset optimizer and node weights before every training iteration
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        end_time = time.time()
        
        training_time = end_time - start_time
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size, 'TW' : sequence_length, 
         'time': training_time}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results003 = pd.concat([results003, pd.DataFrame(d, index=[0])], ignore_index=True)
    results003.to_csv('results/base/fd003.csv')

(21320, 35, 4) (21320, 1) (100, 35, 4)
Epoch 1/20
667/667 [==============================] - 16s 18ms/step - loss: 1808.2336 - val_loss: 688.3946
Epoch 2/20
667/667 [==============================] - 11s 17ms/step - loss: 480.7479 - val_loss: 564.0656
Epoch 3/20
667/667 [==============================] - 11s 16ms/step - loss: 429.9077 - val_loss: 438.1569
Epoch 4/20
667/667 [==============================] - 11s 16ms/step - loss: 410.3593 - val_loss: 550.0168
Epoch 5/20
667/667 [==============================] - 11s 17ms/step - loss: 368.7471 - val_loss: 648.6569
Epoch 6/20
667/667 [==============================] - 11s 16ms/step - loss: 358.0005 - val_loss: 493.2839
Epoch 7/20
667/667 [==============================] - 11s 17ms/step - loss: 344.8911 - val_loss: 417.8543
Epoch 8/20
667/667 [==============================] - 11s 16ms/step - loss: 322.1738 - val_loss: 350.9323
Epoch 9/20
667/667 [==============================] - 11s 17ms/step - loss: 315.0339 - val_loss: 340.7748
Epoch 

In [11]:
results003

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size,TW,time
0,17.544059,0.0,1048.675473,0.0,307.794037,0.0,[64],0.2,tanh,32,35,227.002639
1,15.370333,0.0,1056.089298,0.0,236.247131,0.0,[64],0.2,tanh,32,35,128.112133
2,15.391460,0.0,1027.064005,0.0,236.897064,0.0,[64],0.2,tanh,32,35,93.091394
3,15.892348,0.0,1188.062839,0.0,252.566742,0.0,[64],0.2,tanh,32,35,93.201190
4,15.914256,0.0,1162.119404,0.0,253.263519,0.0,[64],0.2,tanh,32,35,92.851058


### FD001 <a class="anchor" id="fd001"></a>

In [12]:
# Load data
# Load data and preprocess
train, test, y_test = prepare_data('FD001.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']
remaining_sensors = ["Ps30", "Nf", "P30", "BPR"] # selection based on main_notebook 
drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

rul_piecewise = 125
train['RUL'].clip(upper=rul_piecewise, inplace=True)

(20631, 27) (13096, 26) (100, 1)


In [13]:
%%time

# RMSE	std_RMSE	S_score	std_S_score	MSE	std_MSE	alpha	nodes	dropout	activation	batch_size	TW
# 21	13.768992	0.0	3.830708e+02	0.0	189.585159	0.0	0.10	[128]	0.2	tanh	64	30
results001 = pd.DataFrame()

for SEED in range(5):  
    set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    # 0.20	[64]	0.3	tanh	32	25
    
    # parameter's sample
    # weights_file = "weights_file_lstm_optimalmodel_clv.h5"
    alpha = 0.1
    sequence_length = 30
    epochs = 20
    nodes_per_layer = [128]
    dropout = 0.2
    activation = 'tanh'
    batch_size = 64
    remaining_sensors = remaining_sensors
    # create model
    input_shape = (sequence_length, len(remaining_sensors))    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout, activation)
       
    # Model fitting
    with tf.device('/device:GPU:0'):
        start_time = time.time()
        weights_file = model.get_weights()
        model.set_weights(weights_file)  # reset optimizer and node weights before every training iteration
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        end_time = time.time()
        
        training_time = end_time - start_time
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size, 'TW' : sequence_length, 
         'time': training_time}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results001 = pd.concat([results001, pd.DataFrame(d, index=[0])], ignore_index=True)
    results001.to_csv('results/base/fd001.csv')

(17731, 30, 4) (17731, 1) (100, 30, 4)
Epoch 1/20
278/278 [==============================] - 6s 15ms/step - loss: 2094.1931 - val_loss: 1618.0138
Epoch 2/20
278/278 [==============================] - 4s 15ms/step - loss: 1218.1819 - val_loss: 704.2067
Epoch 3/20
278/278 [==============================] - 4s 14ms/step - loss: 671.6970 - val_loss: 451.3782
Epoch 4/20
278/278 [==============================] - 4s 15ms/step - loss: 414.8923 - val_loss: 341.0276
Epoch 5/20
278/278 [==============================] - 4s 14ms/step - loss: 351.7847 - val_loss: 256.1475
Epoch 6/20
278/278 [==============================] - 4s 15ms/step - loss: 309.1059 - val_loss: 297.0459
Epoch 7/20
278/278 [==============================] - 4s 15ms/step - loss: 292.2281 - val_loss: 203.4271
Epoch 8/20
278/278 [==============================] - 4s 15ms/step - loss: 287.8505 - val_loss: 208.9244
Epoch 9/20
278/278 [==============================] - 4s 15ms/step - loss: 272.1981 - val_loss: 199.2759
Epoch 10/20
2

In [14]:
results001

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size,TW,time
0,13.711273,0.0,345.463846,0.0,187.998978,0.0,[128],0.2,tanh,64,30,81.581375
1,14.854724,0.0,364.273577,0.0,220.662811,0.0,[128],0.2,tanh,64,30,82.244397
2,14.248470,0.0,362.494277,0.0,203.018890,0.0,[128],0.2,tanh,64,30,79.451121
3,14.543846,0.0,458.032573,0.0,211.523483,0.0,[128],0.2,tanh,64,30,79.214682
4,14.363899,0.0,437.720490,0.0,206.321594,0.0,[128],0.2,tanh,64,30,80.568281
